In [72]:
%load_ext autoreload
%autoreload 2
import sys
import torch
sys.path.append("src")
from azure_util import AzureBlobStorage

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import torch

class Network(torch.nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self._layers = torch.nn.Sequential(
            torch.nn.Linear(100, 200),
            torch.nn.ELU(),
            torch.nn.Linear(200, 2),
        )
        
    def forward(self, x):
        
        return self._layers(x)
    
network = Network()

In [60]:
import os
PROJECT_NAME = "test"
PROJECT_PATH = os.path.join("models", PROJECT_NAME)
VERSION = "1"
os.makedirs(PROJECT_PATH, exist_ok = True)
MODEL_PATH = os.path.join(PROJECT_PATH, VERSION)
os.makedirs(MODEL_PATH, exist_ok = True)

## Create the config for the model and trace the model


In [61]:

config = '''
name: "test_network"
platform: "pytorch_libtorch"
max_batch_size: 10
input [
  {
    name: "INPUT__0"
    data_type: TYPE_FP32
    dims: [100]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [2]
  }
]

instance_group [
    {
        count: 1
        kind: KIND_GPU
    }
]

dynamic_batching {
  preferred_batch_size: [ 4, 8 ],
  max_queue_delay_microseconds: 1000
}
'''

f = open(os.path.join(PROJECT_PATH, "config.pbtxt"), "w")
f.write(config)
f.close()

In [ ]:
import torch
# Trace the model and save it 
example = torch.rand(1, 100)
traced_script_module = torch.jit.trace(network, example)
traced_script_module.save(os.path.join(MODEL_PATH, "model.pt"))

## Upload the folder containing the config and traced model to azure, triton inference server will later grab the files from there

In [64]:
import os

azure_blob = AzureBlobStorage(
    connection_string=os.environ['AZURE_CHECKPOINT_CONNECTION_STRING'],
    container_name="kfserve2",#, os.environ['AZURE_CHECKPOINT_CONTAINER_NAME'],
)
azure_blob.upload_folder("context", "")

2023-05-18 14:14:15.661 95 azure.core.pipeline.policies.http_logging_policy INFO [on_request():510] Request URL: 'https://deeplearning.blob.core.windows.net/kfserve2/models/test/config.pbtxt'
Request method: 'PUT'
Request headers:
    'Content-Length': '487'
    'x-ms-blob-type': 'REDACTED'
    'x-ms-version': 'REDACTED'
    'Content-Type': 'application/octet-stream'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.16.0 Python/3.10.9 (Linux-5.19.0-38-generic-x86_64-with-glibc2.27)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '455eaee2-f586-11ed-b0c5-0242ac110002'
    'Authorization': 'REDACTED'
A body is sent with the request
2023-05-18 14:14:15.985 95 azure.core.pipeline.policies.http_logging_policy INFO [on_response():549] Response status: 201
Response headers:
    'Content-Length': '0'
    'Content-MD5': 'REDACTED'
    'Last-Modified': 'Thu, 18 May 2023 14:14:16 GMT'
    'ETag': '"0x8DB57AA29E322E2"'
    'Server': 'Windows-Azure-Blob/1.0 